In [1]:
import os  
import glob
import sklearn
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

import PIL 
import random
import numpy as np
import matplotlib.pyplot as plt 

import torch
import torch.nn as nn
import torchvision.models as models
from efficientnet_pytorch import EfficientNet
from torchinfo import summary 

import torch.optim as optim
from IPython.display import Image
from torch.utils.data import DataLoader, Dataset

from torchvision.datasets import ImageFolder
from torchvision.transforms import transforms
import torch.nn.functional as F

import pydicom
from PIL import Image

import cv2

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device 

device(type='cuda')

Set Random Seed for reproducability

In [3]:
random_seed = 124
np.random.seed(random_seed)

torch.manual_seed(random_seed)
torch.backends.cudnn.deterministic = True

In [80]:
train_df = pd.read_csv('C:/Users/Lucas/Documents/PIBIC/DATASET/NIH-CHEST/train_df.csv')
test_df = pd.read_csv('C:/Users/Lucas/Documents/PIBIC/DATASET/NIH-CHEST/test_df.csv')
valid_df = pd.read_csv('C:/Users/Lucas/Documents/PIBIC/DATASET/NIH-CHEST/valid_df.csv')

In [81]:
train_df = train_df[['Image Index', 'Finding Labels', 'Path Image']]
test_df = test_df[['Image Index', 'Finding Labels', 'Path Image']]
valid_df = valid_df[['Image Index', 'Finding Labels', 'Path Image']]

In [82]:
valid_df['Finding Labels'].unique()

array(['No Finding'], dtype=object)

In [74]:
df = valid_df

binary_labels = np.zeros((len(df), 7))
for i, row_label in enumerate(df['Finding Labels']):
    for j, label in enumerate(labels):
        if label == row_label:
            binary_labels[i][j] = 1
            
            
df['testesteste'] = list(binary_labels)
df

,Image Index,Finding Labels,Path Image,testesteste
0,00014396_000.png,No Finding,C:/Users/Lucas/Documents/PIBIC/DATASET/NIH-CHE...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
1,00025595_002.png,No Finding,C:/Users/Lucas/Documents/PIBIC/DATASET/NIH-CHE...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
2,00012520_003.png,No Finding,C:/Users/Lucas/Documents/PIBIC/DATASET/NIH-CHE...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
3,00001274_001.png,No Finding,C:/Users/Lucas/Documents/PIBIC/DATASET/NIH-CHE...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
4,00030596_000.png,No Finding,C:/Users/Lucas/Documents/PIBIC/DATASET/NIH-CHE...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
...,...,...,...,...
1495,00013946_003.png,No Finding,C:/Users/Lucas/Documents/PIBIC/DATASET/NIH-CHE...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
1496,00000854_000.png,No Finding,C:/Users/Lucas/Documents/PIBIC/DATASET/NIH-CHE...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
1497,00029437_003.png,No Finding,C:/Users/Lucas/Documents/PIBIC/DATASET/NIH-CHE...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
1498,00016410_024.png,No Finding,C:/Users/Lucas/Documents/PIBIC/DATASET/NIH-CHE...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"


In [72]:
binary_labels.shape

(1500, 7)

In [68]:
# Assuming df is your DataFrame and "Finding Labels" is your column with the labels
def labels_to_binary(df, labels_column, unique_labels):
    binary_labels = np.zeros((len(df), len(unique_labels)))
    for i, row_label in enumerate(df[labels_column]):
        for j, label in enumerate(unique_labels):
            if label == row_label:
                binary_labels[i][j] = 1
    return binary_labels

labels = ['No Finding', 'Infiltration', 'Atelectasis', 'Effusion', 'Nodule', 'Pneumothorax', 'Mass']
valid_df['Labels'] = list(labels_to_binary(train_df, 'Finding Labels', labels))

ValueError: Length of values (31803) does not match length of index (1500)

In [75]:
train_df['Labels'] = list(labels_to_binary(train_df, 'Finding Labels', labels))
test_df['Labels'] = list(labels_to_binary(test_df, 'Finding Labels', labels))
valid_df['Labels'] = list(labels_to_binary(train_df, 'Finding Labels', labels))


ValueError: Length of values (31803) does not match length of index (1500)

In [79]:
train_path, train_label = list(train_df['Path Image']), list(train_df['Labels'])
test_path, test_label = list(test_df['Path Image']), list(test_df['Labels'])
valid_path, valid_label = list(valid_df['Path Image']), list(valid_df['Labels'])

KeyError: 'Labels'

In [7]:
len(['No Finding', 'Infiltration', 'Atelectasis', 'Effusion', 'Nodule', 'Pneumothorax', 'Mass'])

7

In [8]:
# Define the labels and create a mapping from label to number
labels = ['No Finding', 'Infiltration', 'Atelectasis', 'Effusion', 'Nodule', 'Pneumothorax', 'Mass']
label_to_number = {label: i for i, label in enumerate(labels)}



# Map the labels to numbers
train_label = [label_to_number[label] for label in train_label]
test_label = [label_to_number[label] for label in test_label]
valid_label = [label_to_number[label] for label in valid_label]

In [61]:
train_df.head(1)

,Image Index,Finding Labels,Path Image
0,00000005_006.png,Infiltration,C:/Users/Lucas/Documents/PIBIC/DATASET/NIH-CHE...


### Dataset Class



In [10]:
class CT_Dataset(Dataset):
    def __init__(self, img_path, img_labels, img_transforms=None, grayscale=False):
        self.img_path = img_path
        self.img_labels = torch.Tensor(img_labels)
        if (img_transforms is None) & (grayscale == True):
            self.transforms = transforms.Compose([transforms.Grayscale(),
                                                  transforms.Resize((250, 250)),
                                                  transforms.ToTensor()])
        elif grayscale == False:
            self.transforms = transforms.Compose([transforms.Resize((250, 250)),
                                                  transforms.ToTensor()])
        else:
            self.transforms = img_transforms
    
    def __getitem__(self, index):
        # load image
        cur_path = self.img_path[index]
        cur_img = PIL.Image.open(cur_path).convert('RGB')
        cur_img = self.transforms(cur_img)

        return cur_img, self.img_labels[index]
    
    def __len__(self):
        return len(self.img_path)

## 2. Model Development

### ResNet50

In [11]:

class ResNet50(nn.Module):
    def __init__(self, num_classes):
        super(ResNet50, self).__init__()
        self.resnet50 = models.resnet50(pretrained=True)
        in_features = self.resnet50.fc.in_features
        self.resnet50.fc = nn.Sequential(
            nn.Linear(in_features, 512),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        return self.resnet50(x)


### ResNet101

In [12]:
class ResNet101(nn.Module):
    def __init__(self, num_classes):
        super(ResNet101, self).__init__()
        self.resnet101 = models.resnet101(pretrained=True)
        in_features = self.resnet101.fc.in_features
        self.resnet101.fc = nn.Sequential(
            nn.Linear(in_features, 512),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        return self.resnet101(x)

### EfficientNetB0

In [13]:
class EfficientNetB0(nn.Module):
    def __init__(self, num_classes):
        super(EfficientNetB0, self).__init__()
        self.effnet = EfficientNet.from_pretrained('efficientnet-b0')
        in_features = self.effnet._fc.in_features
        self.effnet._fc = nn.Sequential(
            nn.Linear(in_features, 512),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        return self.effnet(x)


### EfficientNetB4

In [14]:
class EfficientNetB4(nn.Module):
    def __init__(self, num_classes):
        super(EfficientNetB4, self).__init__()
        self.effnet = EfficientNet.from_pretrained('efficientnet-b4')
        in_features = self.effnet._fc.in_features
        self.effnet._fc = nn.Sequential(
            nn.Linear(in_features, 512),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        return self.effnet(x)


### EfficientNetB7

In [15]:
class EfficientNetB7(nn.Module):
    def __init__(self, num_classes):
        super(EfficientNetB7, self).__init__()
        self.effnet = EfficientNet.from_pretrained('efficientnet-b7')
        in_features = self.effnet._fc.in_features
        self.effnet._fc = nn.Sequential(
            nn.Linear(in_features, 512),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        return self.effnet(x)


### VGG16

In [16]:
class VGG16(nn.Module):
    def __init__(self, num_classes):
        super(VGG16, self).__init__()
        self.vgg16 = models.vgg16(pretrained=True)
        in_features = self.vgg16.classifier[6].in_features
        self.vgg16.classifier[6] = nn.Sequential(
            nn.Linear(in_features, 512),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        return self.vgg16(x)


### Function train_model

In [17]:
result_df = pd.read_csv('C:/Users/Lucas/Documents/PIBIC/DATASET/NIH-CHEST/result_df.csv')
qnt_labels = result_df['Finding Labels'].value_counts()
qnt_images_each_label = qnt_labels.to_dict()
qnt_images_each_label

{'No Finding': 15000,
 'Infiltration': 9547,
 'Atelectasis': 4215,
 'Effusion': 3955,
 'Nodule': 2705,
 'Pneumothorax': 2194,
 'Mass': 2139}

In [18]:
total_samples = 0
for value in qnt_images_each_label.values():
    total_samples += value

In [19]:
qnt_images_each_label['Infiltration']

9547

In [20]:
labels = list(train_df['Finding Labels'].unique())
num_classes = len(train_df['Finding Labels'].unique())

weight_No_Finding = total_samples/(qnt_images_each_label['No Finding'] * num_classes)
weight_Infiltration = total_samples/( qnt_images_each_label['Infiltration'] * num_classes)
weight_Atelectasis = total_samples/( qnt_images_each_label['Atelectasis'] * num_classes)
weight_Effusion = total_samples/( qnt_images_each_label['Effusion'] * num_classes)
weight_Nodule = total_samples/( qnt_images_each_label['Nodule'] * num_classes)
weight_Pneumothorax = total_samples/( qnt_images_each_label['Pneumothorax'] * num_classes)
weight_Mass = total_samples/( qnt_images_each_label['Mass'] * num_classes)

weights = [weight_No_Finding, weight_Infiltration, weight_Atelectasis, weight_Effusion, weight_Nodule, weight_Pneumothorax, weight_Mass]
weights

[0.37861904761904763,
 0.5948764757814721,
 1.3473987459752583,
 1.4359761603756547,
 2.0995510958542383,
 2.5885531970308633,
 2.6551125358979495]

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.optim as optim

# Define the training function
def train_model(model, train_dataset, val_dataset, test_dataset, device, 
                lr=0.0001, epochs=30, batch_size=32, l2=0.00001, gamma=0.5,
                patience=7):
    model = model.to(device)

    # Construct dataloader
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

    # History
    history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}

    weight = torch.tensor(weights).to(device)
    
    # Set up loss function and optimizer
    criterion = nn.CrossEntropyLoss(weight=weight)  
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=l2)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=patience, gamma=gamma)

    # Initialize variables to track the best validation loss and corresponding model state
    best_val_loss = float('inf')
    best_model_state = None

    # Training Loop
    print("Training Start:")
    for epoch in range(epochs):
        model.train()

        train_loss = 0
        train_acc = 0
        val_loss = 0
        val_acc = 0

        for i, (images, labels) in enumerate(train_loader):
            images = images.to(device)
            labels = labels.to(device).long()  # Convert labels to Long data type

            outputs = model(images).float()  # Make sure the output is of type float
            pred = outputs.argmax(dim=1)
    
            cur_train_loss = criterion(outputs, labels)
            cur_train_acc = (pred == labels).sum().item() / batch_size

            cur_train_loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            train_loss += cur_train_loss 
            train_acc += cur_train_acc
        
        model.eval()
        with torch.no_grad():
            for images, labels in val_loader:
                images = images.to(device)
                labels = labels.to(device).long()  # Convert labels to Long data type

                outputs = model(images).float()  # Make sure the output is of type float

                cur_valid_loss = criterion(outputs, labels)
                val_loss += cur_valid_loss

                pred = outputs.argmax(dim=1)
                val_acc += (pred == labels).sum().item() / batch_size

        scheduler.step()

        train_loss = train_loss / len(train_loader)
        train_acc = train_acc / len(train_loader)
        val_loss = val_loss / len(val_loader)
        val_acc = val_acc / len(val_loader)

        print(f"Epoch:{epoch + 1} / {epochs}, lr: {optimizer.param_groups[0]['lr']:.5f} train loss:{train_loss:.5f}, train acc: {train_acc:.5f}, valid loss:{val_loss:.5f}, valid acc:{val_acc:.5f}")
    
        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)
    
        # Update the best model if validation loss is the lowest so far
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model_state = model.state_dict()

    # Load the best model state
    if best_model_state is not None:
        model.load_state_dict(best_model_state)

    test_acc = 0
    print(f'The best val loss is {best_val_loss}.\n\n')
    y_true = []
    y_pred = []
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device).long()  # Convert labels to Long data type

            outputs = model(images).float()  # Make sure the output is of type float
            predictions = torch.round(torch.sigmoid(outputs)).cpu().numpy()
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predictions)
            pred = outputs.argmax(dim=1)
            test_acc += (pred == labels).sum().item()
            print(y_true)
            print(y_pred)

    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    
    print(f'Test Accuracy:  {(test_acc / len(test_loader))}')

    return history, model


Process the datasets and train the model

In [22]:
print("Current GPU memory usage:", torch.cuda.memory_allocated() / (1024 ** 2), "MB")
print("Max GPU memory usage:", torch.cuda.max_memory_allocated() / (1024 ** 2), "MB")

torch.cuda.empty_cache()

Current GPU memory usage: 0.0 MB
Max GPU memory usage: 0.0 MB


In [23]:
len(train_path), len(train_label)

(31803, 31803)

In [24]:
train_dataset = CT_Dataset(img_path=train_path, img_labels=np.array(train_label))
val_dataset = CT_Dataset(img_path=valid_path, img_labels=np.array(valid_label))
test_dataset = CT_Dataset(img_path=test_path, img_labels=np.array(test_label))

In [ ]:
# model_resnet50 = ResNet50(num_classes=7)
# train_loader = train_model(model_resnet50, train_dataset, val_dataset, test_dataset, device, lr=0.0002, batch_size= batch_size, epochs=epoch, l2=0.09, patience=5)

### Training

In [ ]:
23035dfbdfbdb

In [25]:
batch_size = 32
epoch = 10
label_model = "ResNet50_"
path_save_model = f'C:/Users/Lucas/Documents/PIBIC/DATASET/NIH-CHEST/model_/{label_model}{epoch}'
# Train the ResNet101 model
model_resnet50 = ResNet50(num_classes=7)
hist_ResNet50, model_ResNet50 = train_model(model_resnet50, train_dataset, val_dataset, test_dataset, device, lr=0.0002, batch_size= batch_size, epochs=epoch, l2=0.09, patience=5)


# # Train the ResNet101 model
# model_resnet101 = ResNet101(num_classes=7)
# hist_ResNet101, model_ResNet101 = train_model(model_resnet101, train_dataset, val_dataset, test_dataset, device, lr=0.0002, batch_size= batch_size, epochs=epoch, l2=0.09, patience=5)


# Train the EfficientNetB0 model
# model_efficientnet_b0 = EfficientNetB
# 
# 333333333333333333333320(num_classes=7)
# hist_EfficientNetB0, model_EfficientNetB0 = train_model(model_efficientnet_b0, train_dataset, val_dataset, test_dataset, device, lr=0.0002, batch_size= batch_size, epochs=epoch, l2=0.09, patience=5)


# Train the EfficientNetB4 model
# model_efficientnet_b4 = EfficientNetB4(num_classes=7)
# hist_EfficientNetB4, model_EfficientNetB4 = train_model(model_efficientnet_b4, train_dataset, val_dataset, test_dataset, device, lr=0.0002, batch_size= batch_size, epochs=epoch, l2=0.09, patience=5)


# # Train the EfficientNetB7 model
# model_efficientnet_b7 = EfficientNetB7(num_classes=7)
# hist_EfficientNetB7, model_EfficientNetB7 = train_model(model_efficientnet_b7, train_dataset, val_dataset, test_dataset, device, lr=0.0002, batch_size= batch_size, epochs=epoch, l2=0.09, patience=5)


# # Train the VGG16 model
# model_vgg16 = VGG16(num_classes=7)
# hist_VGG16, model_VGG16 = train_model(model_vgg16, train_dataset, val_dataset, test_dataset, device, lr=0.0002, batch_size= batch_size, epochs=epoch, l2=0.09, patience=5)model_vgg16 

In [31]:
hist_kernel = hist_ResNet50
model_kernel = model_ResNet50


In [30]:
torch.save(model_resnet50.state_dict(), f'{path_save_model}.pth')

## Metricis

In [ ]:
def plot_metrics(hist, label_model) :
    # plot training curves
    epochs = range(1, len(hist['train_loss']) + 1)

    train_loss = [t.cpu().detach().numpy() for t in hist['train_loss']]
    val_loss = [t.cpu().detach().numpy() for t in hist['val_loss']]


    fig, ax = plt.subplots(1,2, figsize=(20,6))
    ax[0].plot(epochs, train_loss, 'r-', label='Train')
    ax[0].plot(epochs, val_loss, 'b-', label='Evaluation')
    ax[0].set_title('Loss')
    ax[0].set_xlabel('Epochs')
    ax[0].set_ylabel('Loss')
    ax[0].legend()
    plt.savefig(f'{path_save_model}.png')  # This saves the plot as a PNG image

    ax[1].plot(epochs, hist['train_acc'], 'r-', label='Train')
    ax[1].plot(epochs, hist['val_acc'], 'b-', label='Evaluation')
    ax[1].set_title('Accuracy')
    ax[1].set_xlabel('Epochs')
    ax[1].set_ylabel('Acc')
    ax[1].legend()
    plt.savefig(f'{path_save_model}.png')  # This saves the plot as a PNG image
    

    plt.show()
    
    
plot_metrics(hist_kernel, label_model)

In [58]:
def calculate_metrics(model, test_dataset, device, plot_images=False):
    # Initialize variables to store predictions and ground truth
    y_true = []
    y_pred = []
    test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

    cont = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            predictions = torch.round(torch.sigmoid(outputs)).cpu().numpy()
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predictions)
            cont+=1
            if cont == 5:
                break

    # Convert lists to numpy arrays
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)


    return y_true, y_pred



y_true, y_pred = calculate_metrics(model_kernel, test_dataset, device, plot_images=True)

In [59]:
from sklearn.metrics import multilabel_confusion_matrix, precision_score, accuracy_score, recall_score, f1_score


In [60]:
y_true, y_pred

(array([1., 1., 1., 1., 1.], dtype=float32),
 array([[0., 0., 1., 1., 0., 1., 1.],
        [0., 0., 1., 1., 0., 0., 1.],
        [0., 0., 1., 1., 0., 1., 1.],
        [1., 0., 1., 1., 0., 1., 1.],
        [0., 0., 1., 1., 0., 0., 1.]], dtype=float32))

In [48]:
cm_matrix = confusion_matrix(y_true, y_pred)

ValueError: Classification metrics can't handle a mix of binary and multilabel-indicator targets

In [45]:
# Calculate evaluation metrics
cm_matrix = multilabel_confusion_matrix(y_true, y_pred).ravel()
precision = precision_score(y_true, y_pred, average='weighted')
accuracy = accuracy_score(y_true, y_pred)
recall = recall_score(y_true, y_pred, average='weighted')
f1 = f1_score(y_true, y_pred, average='weighted')
specificity = tn / (tn + fp)
sensitivity = tp / (fn + tp)

metrics = {
    'precision': precision,
    'recall': recall,
    'f1_score': f1,
    'accuracy': accuracy,
    'specificity': specificity,
    'sensitivity': sensitivity,
    'false negativo': fn
}


ValueError: Classification metrics can't handle a mix of binary and multilabel-indicator targets

In [40]:
def calculate_metrics(model, test_dataset, device, plot_images=False):
    # Initialize variables to store predictions and ground truth
    y_true = []
    y_pred = []
    test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

    cont = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            predictions = torch.round(torch.sigmoid(outputs)).cpu().numpy()
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predictions)
            cont+=1
            if cont == 5:
                break

    # Convert lists to numpy arrays
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    # Calculate evaluation metrics
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    precision = precision_score(y_true, y_pred, average='weighted')
    accuracy = accuracy_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')
    specificity = tn / (tn + fp)
    sensitivity = tp / (fn + tp)

    metrics = {
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'accuracy': accuracy,
        'specificity': specificity,
        'sensitivity': sensitivity,
        'false negativo': fn
    }

    return metrics



model_path = f'{path_save_model}.pth'

if label_model[:7] == "ResNet1":
    model_kernel = ResNet101(num_classes=7)

elif label_model[:7] == "ResNet5":
    model_kernel = ResNet50(num_classes=7)

elif label_model[:14] == "EfficientNetB0":
    model_kernel = EfficientNetB0(num_classes=7)
    
elif label_model[:14] == "EfficientNetB4":
    model_kernel = EfficientNetB4(num_classes=7)

elif label_model[:14] == "EfficientNetB7":
    model_kernel = EfficientNetB7(num_classes=7)

elif label_model[:5] == "VGG16":
    model_kernel = VGG16(num_classes=7)


model_kernel.load_state_dict(torch.load(model_path))
model_kernel.to(device)  # Move the model to the specified device
model_kernel.eval()  # Set the model to evaluation mode

# Call the function with plot_images=True to plot images
metrics = calculate_metrics(model_kernel, test_dataset, device, plot_images=True)
metrics

c:\Users\Lucas\anaconda3\envs\torch000\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Lucas\anaconda3\envs\torch000\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ValueError: Classification metrics can't handle a mix of binary and multilabel-indicator targets

In [37]:
y_true = ["cat", "ant", "cat", "cat", "ant", "bird"]
y_pred = ["ant", "ant", "cat", "cat", "ant", "cat"]
multilabel_confusion_matrix(y_true, y_pred)



array([[[3, 1],
        [0, 2]],

       [[5, 0],
        [1, 0]],

       [[2, 1],
        [1, 2]]], dtype=int64)

In [ ]:
pd.DataFrame(metrics.items(), columns=['Metric', 'Value']).to_csv(f'{path_save_model}.csv')

In [ ]:
def process_images(model, test_dataset, device, plot_images=False, num_images_to_plot=10):
    # Initialize variables to store predictions and ground truth
    y_pred = []
    test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)
    images_plotted = 0  # Counter for the number of images plotted

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            predictions = torch.round(torch.sigmoid(outputs)).cpu().numpy()
            y_pred.extend(predictions)

            # Plot images with true and predicted labels
            if plot_images and images_plotted < num_images_to_plot:
                plot_image(images, labels, predictions)
                images_plotted += 1

            # Break the loop if the desired number of images is plotted
            if images_plotted >= num_images_to_plot:
                break


def plot_image(images, true_label, predicted_label):
    if len(images.shape) == 4 and images.shape[1] == 3:
        # Convert CUDA tensor to numpy array and rearrange dimensions for RGB image
        images = images.cpu().detach().numpy().squeeze().transpose((1, 2, 0))
    else:
        # Convert CUDA tensor to numpy array and squeeze if necessary
        images = images.cpu().detach().numpy().squeeze()

    if len(images.shape) == 2:
        plt.imshow(images, cmap='gray')
    else:
        plt.imshow(images)
    
    # Format the labels
    true_label = true_label.item() if isinstance(true_label, torch.Tensor) else true_label
    predicted_label = predicted_label.item() if isinstance(predicted_label, torch.Tensor) else predicted_label
    
    plt.title(f'True label: {true_label}; Predicted: {predicted_label}')
    plt.show()
    
    
process_images(model_kernel, test_dataset, device, plot_images=True)

**Sugestão de Próximos Passos**

1. Colocar para salvar o modelo com a menor validation loss.
2. Colocar código para carregar o modelo salvo e fazer predições em imagens do dataset.
3. Criar métricas de acurácia (Accuracy, Precision, Recall, F-score, etc.) para avaliar o dataset de teste com o modelo salvo.

Quando tudo estiver pronto:
4. Criar novos modelos CNN (começar pela ResNet101).
5. Implementar outros modelos.

Quando estiver pronto:
6. Implementar no seu dataset.


Para binário tem que mudar a loss (CrossEntropy), a sigmoid da layer final para softmax

# Materiais e Métodos

## 1. Datasets

Explicação sobre os conjuntos de dados utilizados.

## 2. Modelos

Explicação sobre como funciona um modelo CNN (Convolutional Neural Network) e os parâmetros utilizados.

### 2.1. EfficientNet

### 2.2. ResNet101

...

## 3. Métricas
